In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [5]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [6]:
X = train_df.drop(['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
y = train_df['Survived']
numeric_features = ['Age', 'Fare', 'SibSp', 'Parch']
categorical_features = ['Pclass', 'Sex', 'Embarked']

In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ])

In [8]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [12]:
models = {
    '逻辑回归': LogisticRegression(max_iter=1000, random_state=42),
    '随机森林': RandomForestClassifier(n_estimators=100, random_state=42)
}

In [13]:
results = {}
for name, model in models.items():
    # 创建包含预处理和模型的管道
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

    # 训练模型
    pipeline.fit(X_train, y_train)

    # 预测与评估
    y_pred = pipeline.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)

    # 交叉验证
    cv_scores = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy')

    results[name] = {
        'model': pipeline,
        'accuracy': accuracy,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std()
    }

In [14]:
best_model_name = max(results, key=lambda x: results[x]['cv_mean'])
best_model = results[best_model_name]['model']
test_X = test_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
test_predictions = best_model.predict(test_X)

In [15]:
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': test_predictions
})

In [ ]:
# 简答题：
# 1. 精确率是什么，评估分类的性能为什么不能只用精确率
#精确率仅关注预测为正类的样本质量，忽略了预测为负类的样本,在不平衡数据中可能产生误导，即使全部预测为负类，精确率也可能很高，但模型毫无实用价值。需结合召回率等指标全面评估，尤其是在不同业务场景中
# 2. 简述下混淆矩阵是什么
#是一个N×N的矩阵（N为类别数），用于展示分类模型的预测结果与真实标签的对应关系，直观反映模型对各类别的分类效果
# 3. 简述下各个性能指标的意思： 准确率，召回率，F1分数，假阳性，PR曲线，ROC曲线，AUC分数
#准确率（Accuracy）：所有预测正确的样本占总样本的比例，适用于平衡数据集，不适用于不平衡数据。
# 召回率（Recall）：又称灵敏度（Sensitivity），真正例占所有真实正类的比例
# F1 分数（F1-Score）：精确率和召回率的调和平均，综合两者表现，适用于不平衡数据，值越接近 1 越好。
# 假阳性（False Positive, FP）：真实为负类但被预测为正类的样本，反映 “误报” 情况（如健康人被诊断为患病）。
# PR 曲线（Precision-Recall Curve）：以召回率为横轴、精确率为纵轴的曲线，展示不同阈值下模型的精确率 - 召回率权衡，适用于不平衡数据。
# ROC 曲线（Receiver Operating Characteristic Curve）：以假正例率（FPR=FP/(FP+TN)）为横轴、真正例率（TPR = 召回率）为纵轴的曲线，展示模型区分正负类的能力，对不平衡数据较稳健。
# AUC 分数（Area Under ROC Curve）：ROC 曲线下的面积，取值范围 [0,1]，值越大表示模型区分能力越强（0.5 为随机猜测，1 为完美分类）
# 4. 简述下准确率-召回率权衡
#准确率 - 召回率权衡指在分类模型中，调整预测阈值时，精确率和召回率通常呈现反向变化的关系：
# 提高阈值：精确率上升（预测为正的样本更可能是真的），但召回率下降。
# 降低阈值：召回率上升（更多真实正类被检出），但精确率下降。
# 5. 如何用二元分类器 去解决多元分类的问题
#一对一，一对多，多标签
# 6. 什么是 多标签-多分类问题？
#多标签 - 多分类问题是指样本可能同时属于多个标签，且类别总数N≥2的分类任务

